## Temperature Ramp (5.0 -> 5.2)

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

t_0 = 5.0;
t_1 = 5.2;
p_max  = 1.4;
steps_ramp = 1e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_equilibrium.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 5.0, tauP = 0.5, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [3]:
hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 955146 / 1055000 | TPS 14.547 | ETA 01:54:24
Time 00:00:20 | Step 955311 / 1055000 | TPS 16.3864 | ETA 01:41:23
Time 00:00:30 | Step 955477 / 1055000 | TPS 16.4251 | ETA 01:40:59
Time 00:00:40 | Step 955633 / 1055000 | TPS 15.5506 | ETA 01:46:29
Time 00:00:50 | Step 955788 / 1055000 | TPS 15.3435 | ETA 01:47:46
Time 00:01:00 | Step 955938 / 1055000 | TPS 14.9477 | ETA 01:50:27
Time 00:01:10 | Step 956049 / 1055000 | TPS 11.0399 | ETA 02:29:23
Time 00:01:20 | Step 956151 / 1055000 | TPS 10.1807 | ETA 02:41:49
Time 00:01:30 | Step 956304 / 1055000 | TPS 15.2423 | ETA 01:47:55
Time 00:01:40 | Step 956458 / 1055000 | TPS 15.3552 | ETA 01:46:57
Time 00:01:50 | Step 956587 / 1055000 | TPS 12.8717 | ETA 02:07:25
Time 00:02:00 | Step 

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x10762f668>),
             ('particles', <hoomd.data.particle_data at 0x10ac38128>),
             ('number_density', 0.7297366814412944),
             ('bonds', <hoomd.data.bond_data at 0x10ac38208>),
             ('angles', <hoomd.data.angle_data at 0x10ac38278>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x10ac382e8>),
             ('impropers', <hoomd.data.dihedral_data at 0x10ac38358>),
             ('constraints', <hoomd.data.constraint_data at 0x10ac383c8>),
             ('pairs', <hoomd.data.bond_data at 0x10ac38438>),
             ('timestep', 1055000)])